In [7]:
%matplotlib inline

import os 
import sys 
import time
import hashlib

from datetime  import datetime 

import numpy as np  # learn 
import pandas as pd # learn
from pandas import *
from numpy import *

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *

 
import matplotlib as mplt # learn matplolib 
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

import seaborn as sns 
sns.set_style("whitegrid")
sns.set(rc={"figure.figsize": (14, 6)})

import IPython
from IPython.display import display


import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
 
import sklearn as sk
import itertools


from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from pyspark.mllib.regression import LabeledPoint
import random
import time
import copy


In [ ]:
#BT data loading

# dta_RDD = sc.textFile("file:///home/tguo/data/tian-test/udpjitter-H1april2014_regTree.csv")

dta_RDD = sc.textFile("hdfs://computer61.ant-net/user/tguo/udpjitter-H1april2014_regTree_mllib.csv")


dta_splited = dta_RDD.map(lambda line: line.split(",")).map(lambda r:
                                                                (float(r[0]),int( float(r[1])), 
                                                                 int(float(r[2])),int(float(r[3])),
                                                                 int(float(r[4])),int(float(r[5])),
                                                                 int(float(r[6])),int(float(r[7])) ))
# dta_splited.first()

dta= dta_splited
dta.cache()
print dta.first()
print dta.count()


# dta_filtered = dta.filter(lambda line: line[0] <2000 )
# dta_filtered.cache()
# dta_filtered.count()
# print dta.count()

In [2]:
# synthetic data
dta_RDD = sc.textFile("file:///home/tguo/tian_src/synthetic_data.txt")
dta_RDD.cache()

dta_splited = dta_RDD.map(lambda line: line.split(",")).map(lambda r: ( float(r[3]),int(r[0]),int(r[1]),int(r[2])) )
# ( float(r[2]),float(r[0]),float(r[1]) ) 

dta= dta_splited
dta.cache()

dta = dta.map(lambda line: ( line[0]+5, line[1],line[2],line[3] )  )

print dta.first()
print dta.count()


(51.7446171548, 4, 13, 105)
1000000


In [12]:
#statistical on synthetic data

val_interval=set()

localY = dta.map( lambda line: line[0]).collect()
cnt= len(localY)

for i in range(0,cnt ):
    


[(10004.27378586, 5, 15, 105),
 (10004.73888607, 4, 15, 101),
 (10003.95209524, 5, 11, 103),
 (10005.3538609, 4, 15, 105),
 (10005.4505216, 4, 14, 101),
 (10005.4604178, 5, 14, 103),
 (10005.0021227, 5, 13, 104),
 (10004.91370412, 6, 14, 105),
 (10006.9048131, 4, 11, 101),
 (10004.81254134, 4, 12, 104)]

In [ ]:
# test c-approxmate hist

# for i in range(16,32):
print dta.filter(lambda line: line[0]<0).collect()

print cal_bin_cAppHist(-10.2, 1.05)


In [15]:
# basic functions in one-pass method [on the cluster side]

# assign each data instance to the node
def search_nodeToData(features, tree):
        
    nodeNum=len(tree)
    if nodeNum == 0:
        return 0;
    
    current_nodeIdx=0
    while current_nodeIdx< nodeNum:
        split_feature= tree[current_nodeIdx][0]
        split_valueSet= tree[current_nodeIdx][1]
        
        if split_feature==-1:
            return -1
        
        if (( 1<<features[split_feature]) &  split_valueSet) != 0 :
            current_nodeIdx=current_nodeIdx*2+1
        else:
            current_nodeIdx=current_nodeIdx*2+2
      
    return current_nodeIdx - nodeNum

def ini_newBin_cAppHist(hist,binId):
    hist.update( {binId:[1]} )
    
def cal_bin_cAppHist(Y, cratio):
#     lower-bound as the bin index
#     print Y
    return int(math.floor( math.log10(Y)*1.0 / math.log10(cratio)  ))
    
def update_cAppHist(hist, binId):
    hist_local= hist 
    
    if binId in hist.keys():
        hist[binId][0]=hist[binId][0]+1
    else:
        ini_newBin_cAppHist(hist,binId)
    return hist_local

def merge_cAppHist(hist1, hist2):
    hist_local= hist1
    
    for k in hist_local.keys():
        if k in hist2.keys():
            hist_local[k][0]= hist_local[k][0]+ hist2[k][0]
    
    for k in hist2.keys():
        if k not in hist_local.keys():
            hist_local.update({k:hist2[k]})
    return hist_local


def partition_combiner_cAppHist(list_dvAndfeatures):
    
    nodes_dict={}
    tmpcnt=0
    
    for dvAndfeatures in list_dvAndfeatures:
        
        Y= dvAndfeatures[0]
        binId= cal_bin_cAppHist(Y, cApp_ratio)
        node= search_nodeToData(dvAndfeatures[1:numFeatures+1], \
                                currentNode_split_fromMaster.value)
        
        if node in nodes_dict.keys():
            
            # new added: sum of y in a node
            nodes_dict[node]['sumY'] = nodes_dict[node]['sumY']+Y 
            nodes_dict[node]['count'] = nodes_dict[node]['count']+1 
            update_cAppHist(nodes_dict[node]['hist'], binId)
            
            for i in range(0,numFeatures):    
                feature_val= dvAndfeatures[i+1]
                
                if feature_val not in nodes_dict[node][i]:
              
                    nodes_dict[node][i].update( { feature_val: {}   }  )
                    nodes_dict[node][i][feature_val].update( {'count_inFeatureValue':1} )
                    nodes_dict[node][i][feature_val].update( {'hist_inFeatureValue': {} } )  
                    ini_newBin_cAppHist(nodes_dict[node][i][feature_val]['hist_inFeatureValue'], binId)
                    
                else:
                    nodes_dict[node][i][feature_val]['count_inFeatureValue'] = \
                    nodes_dict[node][i][feature_val]['count_inFeatureValue']+1     
                    update_cAppHist(nodes_dict[node][i][feature_val]['hist_inFeatureValue'], binId)
        else:
            nodes_dict.update( {node: {}})  
            # new added: sum of y in a node
            nodes_dict[node].update( { 'sumY': Y} )
            nodes_dict[node].update( { 'count': 1} )
            nodes_dict[node].update( { 'hist': {}} )
            ini_newBin_cAppHist(nodes_dict[node]['hist'],binId)
        
            for i in range(0,numFeatures):
                feature_val= dvAndfeatures[i+1]
                
                nodes_dict[node].update( { i: {}} )    
                nodes_dict[node][i].update( { feature_val: {}   }  )
                nodes_dict[node][i][feature_val].update( {'count_inFeatureValue':1} )
                nodes_dict[node][i][feature_val].update( {'hist_inFeatureValue': {} } )
                
                ini_newBin_cAppHist(nodes_dict[node][i][feature_val]['hist_inFeatureValue'], binId)      
                        
    return zip(nodes_dict.keys(), nodes_dict.values())

def merge_parttion_combiner_cAppHist(nodeToFeatureToValue_1,nodeToFeatureToValue_2 ):
# optimization: calculate median and amd for feature-value    
    
    nodeToFeatureToValue_1['sumY']= nodeToFeatureToValue_1['sumY']+\
    nodeToFeatureToValue_2['sumY']
    nodeToFeatureToValue_1['count']= nodeToFeatureToValue_1['count']+\
    nodeToFeatureToValue_2['count'] 
    nodeToFeatureToValue_1['hist']=merge_cAppHist(nodeToFeatureToValue_1['hist'],nodeToFeatureToValue_2['hist'])
    
    for i in range(0, numFeatures): #feature
        for j in nodeToFeatureToValue_1[i].keys(): #feature value
            
            feature_val=j
            
            if feature_val in nodeToFeatureToValue_2[i].keys():
                nodeToFeatureToValue_1[i][j]['hist_inFeatureValue'] = \
                merge_cAppHist(nodeToFeatureToValue_1[i][j]['hist_inFeatureValue'],\
                               nodeToFeatureToValue_2[i][j]['hist_inFeatureValue'])
                nodeToFeatureToValue_1[i][j]['count_inFeatureValue'] = \
                    nodeToFeatureToValue_1[i][j]['count_inFeatureValue']+ \
                    nodeToFeatureToValue_2[i][j]['count_inFeatureValue']
                
    for i in range(0, numFeatures):
        for j in nodeToFeatureToValue_2[i].keys():
            feature_val=j
            if feature_val not in nodeToFeatureToValue_1[i].keys():
                nodeToFeatureToValue_1[i].update({feature_val: {} })
                nodeToFeatureToValue_1[i][feature_val]=nodeToFeatureToValue_2[i][feature_val].copy()
    return  nodeToFeatureToValue_1


In [13]:
#debug for median and mean absolute deviation 

tmphist= { 1:[100], 2:[30], 5:[22]   }

print MAD_cAppHist( tmphist, 152 )

(1.0895049504950496, 7.6651430462562296)


In [11]:
#debug for median and mean absolute deviation 
print math.pow(1.05,1)
print math.pow(1.05,2)
print math.pow(1.05,5)

1.05
1.1025
1.2762815625


In [19]:
# split decision [on the local side]

#implementation notes:
# bit-operation to record the set of selected feature values in the left child

#remove hist2 from hist1
def minus_local_cAppHist(hist1, hist2):
    
    hist = copy.deepcopy(hist1)
    for i in hist2.keys():
        hist[i][0]= hist[i][0]-hist2[i][0]
    return hist
    
def merge_local_cAppHist(hist1, hist2):
    hist_local= copy.deepcopy(hist1)  
    
    for k in hist_local.keys():
        if k in hist2.keys():
            hist_local[k][0]= hist_local[k][0]+ hist2[k][0]
    
    for k in hist2.keys():
        if k not in hist_local.keys():
            hist_local.update({k:hist2[k]})
    return hist_local

#MAD MSE Trimmed_MSE
#MAD: median absolute median in a histogram   
def MAD_cAppHist(hist, cnt):
    
    sorted_hist= sorted(hist.items(), key= lambda line: line[0])
    num_bins= len(hist)
    
    cntByBounds=0
    curCnt=0
    
    mid_bin=0
    midCnt= cnt/2.0
    flag=0
    cnt_beforeMedian=0
    
    meanAbsDevi=0
    median=0
    
    for i in range(0, num_bins): 
        
        if flag==0:
            
            tmp_curCnt=curCnt+ sorted_hist[i][1][0]
            tmp_cntByBounds =cntByBounds - ( math.pow(cApp_ratio, sorted_hist[i][0])*(cApp_ratio+1))/2.0\
            *sorted_hist[i][1][0]
        
            if tmp_curCnt >= midCnt:
                flag=1
                mid_bin= i
                cnt_beforeMedian = curCnt
                
            else:    
                curCnt=tmp_curCnt
                cntByBounds=tmp_cntByBounds
        else:
            curCnt= curCnt - sorted_hist[i][1][0]
            cntByBounds= cntByBounds + \
            ( math.pow(cApp_ratio, sorted_hist[i][0])*(cApp_ratio+1) )/2.0*sorted_hist[i][1][0]

    if cnt == 1:
        return ( ( math.pow(cApp_ratio, sorted_hist[i][0])*(cApp_ratio+1))/2.0  ,0) 
    elif cnt ==0:
        return (0,0)
    else:
        sample_inMedBin = midCnt - cnt_beforeMedian
        median= math.pow(cApp_ratio, sorted_hist[mid_bin][0])+\
        ( math.pow(cApp_ratio, sorted_hist[mid_bin][0]+1)-math.pow(cApp_ratio, sorted_hist[mid_bin][0]) )/(sorted_hist[ mid_bin][1][0]+1)*sample_inMedBin 
    
        meanAbsDevi =  cntByBounds + curCnt*median
        
        #debug
#         print len(beforeMed)
#         print 'MAD before median bin', beforeMed[0]*med + beforeMed[1]*bin_width
        
        sample_cnt = sorted_hist[mid_bin][1][0]
        sample_interval=(math.pow(cApp_ratio, sorted_hist[mid_bin][0]+1)-math.pow(cApp_ratio, sorted_hist[mid_bin][0]) )/\
        (sorted_hist[ mid_bin][1][0]+1)
    
        sample_val= math.pow(cApp_ratio, sorted_hist[mid_bin][0])
       
        for i in range(0, sample_cnt):
            sample_val= sample_val + sample_interval
            meanAbsDevi= meanAbsDevi + abs( sample_val-median)
        
        return (median,meanAbsDevi)


def split_onOneFeature_cAppHist(values,count_node,hist_node,meanAbsDevi_node):
        
    bestSplitMetric =  meanAbsDevi_node 

    #this is a bit-record variable
    leftSplit_valueSet =0 
    leftSplit_count=0
    
    leftSplit_hist= {}
    rightSplit_hist={}
        
    bestLeft=(-1,-1)
    bestRight=(-1,-1)

    #sort feature values according to median 
    sorted_value_map=[]
     
    for i in values.keys():
        currentVal_hist_count= values[i]['count_inFeatureValue']
        currentVal_hist= values[i]['hist_inFeatureValue']
        tmpMedian= MAD_cAppHist(currentVal_hist,currentVal_hist_count)
        sorted_value_map.append(( tmpMedian[0],i ))
            
    sorted_value_map.sort()
    values_cnt= len(sorted_value_map)
    
    node_hist_backup= copy.deepcopy(hist_node)
    
    # scan the sorted feature values
    for k in range(0,values_cnt-1):
        
        current_feature_value=sorted_value_map[k][1]
        currentVal_hist_count= values[current_feature_value]['count_inFeatureValue']
        currentVal_hist= values[current_feature_value]['hist_inFeatureValue']
        
       
        
        #prepare the histograms for left and right splits
        leftSplit_count = leftSplit_count+currentVal_hist_count
        # histograms for the left values
        leftSplit_hist=merge_local_cAppHist( leftSplit_hist, currentVal_hist, )
        # histograms for the right values
        rightSplit_hist= minus_local_cAppHist(node_hist_backup, leftSplit_hist )
    
        left = MAD_cAppHist( leftSplit_hist, leftSplit_count)
        right= MAD_cAppHist( rightSplit_hist, (count_node - leftSplit_count))
        
        leftMedian= left[0]
        leftMetric= left[1]    
        rightMedian= right[0]
        rightMetric= right[1]
        
        #debug
        tmpcnt=0
        for j in leftSplit_hist.keys():
            tmpcnt = tmpcnt + leftSplit_hist[j][0]
        if tmpcnt != leftSplit_count:
            print '!!!! problem in left split'
            
        tmpcnt=0
        for j in rightSplit_hist.keys():
            tmpcnt = tmpcnt + rightSplit_hist[j][0]
        if tmpcnt != (count_node - leftSplit_count):
            print '!!!! problem in right split'
        
        
        #debug
        if leftMedian<0 or rightMedian<0 or leftMetric <=0 or rightMetric <=0:
            print '$$$$ problem in MAD calculation'
        
        
       
        
        current_splitMetric=1.0*leftSplit_count/count_node*leftMetric + \
                               1.0*(count_node - leftSplit_count)/count_node*rightMetric
        
        if current_splitMetric < bestSplitMetric:
            
            bestSplitMetric=current_splitMetric  
            
            bestLeft=left
            bestRight=right
            
            #bit variable to record which value is chosen for the left child
            leftSplit_valueSet=leftSplit_valueSet+ (1<<current_feature_value)        
    
    return (leftSplit_valueSet, [bestLeft[0], bestRight[0], bestSplitMetric  ] )


def find_bestSplit_cAppHist(local_aggre_nodes, current_NumNodes, nodes_tree, \
                        nodes_tree_test):
    
#     nodes_layer=[]
    current_idx_nodes =0
    
    if local_aggre_nodes[ current_idx_nodes ][0] == -1:
        current_idx_nodes=1
    
    #debug 
    tmpnode_cnt=[]
    presplit=[]

    
    for i in range(0, current_NumNodes):
        if local_aggre_nodes[ current_idx_nodes ][0] != i:
#             nodes_layer.append( (-1,-1) )
            nodes_tree.append( (-1,-1) )
            continue

        node_count= local_aggre_nodes[i][1]['count'] 
        node_hist=local_aggre_nodes[i][1]['hist'] 
        
        
        best_splitMetric_ini= MAD_cAppHist( node_hist, node_count)
        best_split_sofar= best_splitMetric_ini[1]
        preSplitMetric_node= best_splitMetric_ini[1]
        best_split=(-1,[-1,-1,-1])
        
        #debug
        tmpnode_cnt.append(node_count)
        presplit.append( best_splitMetric_ini[1]/100000 )
        
        best_split_feature=-1
        best_split_featureValueSet=-1
        
        for j in range(0,numFeatures):
            
            statisticToValues = local_aggre_nodes[i][1][j] 
            split=split_onOneFeature_cAppHist(statisticToValues,node_count, node_hist,preSplitMetric_node)
            
            if split[1][2] < best_split_sofar:
                best_split=split
                best_split_sofar=split[1][2]
                best_split_feature = j
                best_split_featureValueSet= split[0]
   
        
        # for one-layer point assignment
#         nodes_layer.append( (best_split_feature,  best_split_featureValueSet  )  )
        
        # for whole-tree point assignment
        nodes_tree.append(  (best_split_feature,  best_split_featureValueSet  )      )
        
        # tree for predicting
        nodes_tree_test.append(best_split[1][0])
        nodes_tree_test.append(best_split[1][1])
        
        current_idx_nodes= current_idx_nodes+1
    
    #debug
#     print tmpnode_cnt
#     print presplit
    
#     return nodes_layer
        


In [20]:
# check function for distributed information collection

def statistic_infor_check(local_aggre_nodes ):
    
    numNode = len(local_aggre_nodes)

#     print numNode
    tmpsum=0
    for i in range(0, numNode):
        tmpsum=tmpsum+local_aggre_nodes[i][1]['count']
    if tmpsum != dta_train_cnt:
        print '!!!!!! problem in node count'


    tmpsum=0
    for i in range(0, numNode):
        #check node hist
        node_count = local_aggre_nodes[i][1]['count']
        tmpsum=0
        for j in local_aggre_nodes[i][1]['hist']:
            tmpsum= tmpsum+local_aggre_nodes[i][1]['hist'][j][0]
        if tmpsum!= node_count:
            print '$$$$$ problem in histogram of node'
            
        #check feature values
        for j in range(0, numFeatures):
            tmpsum=0
            for k in local_aggre_nodes[i][1][j].keys():
                value_hist= local_aggre_nodes[i][1][j][k]['hist_inFeatureValue']
                value_cnt = local_aggre_nodes[i][1][j][k]['count_inFeatureValue']
                
                tmpsum= tmpsum+value_cnt
            
                #check feature-value hist
                tmpsum1=0
                for m in value_hist.keys():
                    tmpsum1= tmpsum1 + value_hist[m][0]
            
                if tmpsum1 != value_cnt:
                    print '+++++++++++ problem in feature-value histogram', tmpsum1, value_cnt
            
            
            if tmpsum != node_count:
                print '------problem !!!! in feature:', tmpsum, node_count    
   

In [21]:
# single round of training  process
# current_layer=1hist

#without outliers
dta= dta.filter(lambda line: line[0]<1000)
dta_train = dta.sample(False, .7, 12345)
dta_test = dta.sample(False, .3, 43243)
dta_train_cnt = dta_train.count()

maxdepth=7
numFeatures=3
cApp_ratio = 1.05

# dta_train = dta.sample(False, .7, 12345)
# dta_test = dta.sample(False, .3, 43243)
# dta_test = dta_test.filter(lambda line: line[0] <1000 )
# dta_train_cnt = dta_train.count()


nodeSplits_tree=[]
node_tree_test=[]

#ini parameter
currentNode_split_fromMaster = sc.broadcast([])


for i in range(0,maxdepth):
    
    start = time.time()
    
    current_NumNodes= int( math.pow(2,  i)) 
    print 'current split decision:',currentNode_split_fromMaster.value
    
    
      #----------------------- debug ---------------
    
#     error_partitions = dta_train.mapPartitions( error_partition_combiner_hist )

#     error_nodes =error_partitions.reduceByKey(lambda statis_partition_1, \
#                                               statis_partition_2: 
#                               error_merge_parttion_combiner_hist(statis_partition_1,\
#                                                       statis_partition_2 ))
    
#     local_error_nodes= error_nodes.collect()
    
#     local_sorted_error_nodes = sorted(local_error_nodes,key= lambda val:val[0])
    
# #     print 'current number of nodes:', len(local_sorted_error_nodes)
#     tmpres=[]
#     tmpcnt=[]
#     for k in range(0, len(local_sorted_error_nodes)):   
            
#         tmpvar=local_sorted_error_nodes[k][1]['sumYsquare']/local_sorted_error_nodes[k][1]['count']\
#         -(local_sorted_error_nodes[k][1]['sumY']/local_sorted_error_nodes[k][1]['count'])*\
#         (local_sorted_error_nodes[k][1]['sumY']/local_sorted_error_nodes[k][1]['count'])
        
#         tmpres.append( tmpvar/ 100000.0 )
#         tmpcnt.append(local_sorted_error_nodes[k][1]['count'])
        
#     print 'before split:',tmpres
#     print 'before split:', tmpcnt
    
    #---------------------------------------------
    
    
    statis_partitions = dta_train.mapPartitions( partition_combiner_cAppHist )
    aggre_nodes =statis_partitions.reduceByKey(lambda statis_partition_1, statis_partition_2: 
                              merge_parttion_combiner_cAppHist(statis_partition_1,statis_partition_2 ))
    
    local_aggre_nodes= aggre_nodes.collect()
    
    cluster_end=time.time()
    
    #debug
#     statistic_infor_check(local_aggre_nodes)
    
    
    nodeSplits_layer=find_bestSplit_cAppHist(sorted(local_aggre_nodes,key= lambda val:val[0])\
                                    ,current_NumNodes,nodeSplits_tree,node_tree_test)
    
    
    print i,'-th level ----------------------- '
    
    
    #debug
#     statistic_infor_check(local_aggre_nodes)
    
    
    #for whole tree
    currentNode_split_fromMaster = sc.broadcast(nodeSplits_tree)
    
    end = time.time()
    elapsed = end - start

    print i,'-th level running time: ', cluster_end - start,'sec', end- cluster_end, 'sec'
    
    
    #----------------------- debug ---------------
    
#     error_partitions = dta_train.mapPartitions( error_partition_combiner )

#     error_nodes =error_partitions.reduceByKey(lambda statis_partition_1, \
#                                               statis_partition_2: 
#                               error_merge_parttion_combiner(statis_partition_1,\
#                                                       statis_partition_2 ))
    
#     local_error_nodes= error_nodes.collect()
   
    
#     local_sorted_error_nodes = sorted(local_error_nodes,key= lambda val:val[0])
    
# #     print 'current number of nodes:', len(local_sorted_error_nodes)
#     tmpres=[]
#     tmpcnt=[]
    
#     for k in range(0, len(local_sorted_error_nodes)):
        
#         tmpweight=0.0
        
#         if len(local_sorted_error_nodes) > 1:
#             if (k%2) == 0 :
#                 tmpweight= local_sorted_error_nodes[k][1]['count']+ \
#                 local_sorted_error_nodes[k+1][1]['count']
#             else:
#                 tmpweight= local_sorted_error_nodes[k][1]['count']+ \
#                 local_sorted_error_nodes[k-1][1]['count']
#             weight=1.0*local_sorted_error_nodes[k][1]['count']/tmpweight
            
#         else:
#             weight=1.0    
            
# #       weight=1.0     
            
#         tmpvar=local_sorted_error_nodes[k][1]['sumYsquare']/local_sorted_error_nodes[k][1]['count']\
#         -(local_sorted_error_nodes[k][1]['sumY']/local_sorted_error_nodes[k][1]['count'])*\
#         (local_sorted_error_nodes[k][1]['sumY']/local_sorted_error_nodes[k][1]['count'])
        
#         tmpvar= weight*tmpvar/ 100000.0
        
#         tmpres.append( tmpvar )
#         tmpcnt.append( local_sorted_error_nodes[k][1]['count'] )
    
#     reduced_error=[]
#     for k in range(0, len(local_sorted_error_nodes),2):
#         reduced_error.append( tmpres[k]+tmpres[k+1]    )
        
#     print 'after split:',reduced_error
#     print 'after split:',tmpcnt
    
    #---------------------------------------------
    
    #for layer of nodes
    #currentNode_split_fromMaster = sc.broadcast(nodeSplits_layer)



    
    

current split decision: []
0 -th level ----------------------- 
0 -th level running time:  5.71322894096 sec 0.133812904358 sec
current split decision: [(2, 70988433612780846483815379501056L)]
1 -th level ----------------------- 
1 -th level running time:  6.06703186035 sec 0.174139976501 sec
current split decision: [(2, 70988433612780846483815379501056L), (1, 45056), (0, 40)]
2 -th level ----------------------- 
2 -th level running time:  6.02945184708 sec 0.187134981155 sec
current split decision: [(2, 70988433612780846483815379501056L), (1, 45056), (0, 40), (0, 28), (1, 16384), (0, 8), (1, 49152)]
3 -th level ----------------------- 
3 -th level running time:  5.99491810799 sec 0.196682929993 sec
current split decision: [(2, 70988433612780846483815379501056L), (1, 45056), (0, 40), (0, 28), (1, 16384), (0, 8), (1, 49152), (0, 20), (1, 32768), (0, 36), (0, 24), (1, 38912), (1, 24576), (0, 68), (2, 5070602400912917605986812821504L)]
4 -th level ----------------------- 
4 -th level runn

In [24]:
leaf_nodes = node_tree_test[ len(node_tree_test)-  (int)( math.pow(2,maxdepth)): len(node_tree_test)  ]
print leaf_nodes


[11.994305061338633, 23.009706257677312, 11.655536813488723, 69.84793457774158, 33.970533614922516, 48.492409983298316, 46.046167205380634, 89.03330229508093, -1, -1, 37.9673170956686, 41.078971578491355, 41.92198902333693, 50.91803323388012, 89.87058265734697, 98.61105368984497, 23.984570496636056, 25.993116258902663, 24.99546743026356, 36.02168009938599, -1, -1, -1, -1, 29.941250991151218, 48.13159447816529, 54.63663977198971, 62.56559834309815, 49.572230304274434, 92.32288789196225, 75.4566953990882, 91.10198522909808, 7.994789188044853, 14.99877063084032, 19.992255293070812, 24.047430752148493, -1, -1, -1, -1, 5.983774730128319, 60.607909301800106, 24.00897032674882, 56.00933093356765, -1, -1, 100.50802056518128, 100.62802027357161, -1, -1, 48.14416054764111, 48.81883602817873, -1, -1, 64.37665583255757, 78.85825987624672, 46.01924255221292, 46.89233395412004, -1, -1, 28.047808227035826, 71.2929409750476, 69.35418159504353, 80.77412009636357, 14.001900888024508, 29.028258838286746,

In [16]:
# training and testing process: main process 

#with outliers
# dta_train = dta.sample(False, .7, 12345)
# dta_test = dta.sample(False, .3, 43243)
# dta_test = dta_test.filter(lambda line: line[0] <1000 )

#without outliers
dta= dta.filter(lambda line: line[0]<1000)
dta_train = dta.sample(False, .7, 12345)
dta_test = dta.sample(False, .3, 43243)
# dta_test = dta_test.filter(lambda line: line[0] <1000 )

maxdepth=10
numFeatures=3
cApp_ratio = 1.05


def search_nodeToData(features, tree):
    nodeNum=len(tree)   
    if nodeNum == 0:
        return 0;
    
    current_nodeIdx=0
    while current_nodeIdx< nodeNum:
        split_feature= tree[current_nodeIdx][0]
        split_valueSet= tree[current_nodeIdx][1]
        
        if split_feature==-1:
            return -1
        if (( 1<<features[split_feature]) &  split_valueSet) != 0 :
            current_nodeIdx=current_nodeIdx*2+1
        else:
            current_nodeIdx=current_nodeIdx*2+2        
    return current_nodeIdx - nodeNum

def tree_test_mapFunc(line):
    tmpnode = search_nodeToData(line[1:numFeatures+1], nodeSplits_tree)
    return (leaf_nodes[ tmpnode ]  - line[0])*(leaf_nodes[ tmpnode ]  - line[0])
    
def tree_test( testData_rdd ):
    err_rdd=testData_rdd.map( lambda line: tree_test_mapFunc(line))
    
    #debug
#     print err_rdd.take(20)
    
    err_sum = err_rdd.reduce( lambda a,b: a+b)
    return err_sum / testData_rdd.count()


#debug
def debug_tree_test_mapFunc(line):
    tmpnode = search_nodeToData(line[1:numFeatures+1], nodeSplits_tree)
    return (leaf_nodes[ tmpnode ], line[0])
    
def debug_tree_test( testData_rdd ):
    err_rdd=testData_rdd.map( lambda line: debug_tree_test_mapFunc(line))
#     print err_rdd.take(20)
    
    
#ini parameter on the cluster
currentNode_split_fromMaster = sc.broadcast([])
test_err = []
train_err = []
run_time=[]

for i in range(1,maxdepth):
    nodeSplits_tree=[]
    node_tree_test=[]
    currentNode_split_fromMaster = sc.broadcast([])
    
    start = time.time()
    
    for cur_depth in range(0,i):
        
        current_NumNodes= int( math.pow(2,  cur_depth)) 
        
        statis_partitions = dta_train.mapPartitions( partition_combiner_cAppHist )
        aggre_nodes =statis_partitions.reduceByKey(lambda statis_partition_1, statis_partition_2: 
                              merge_parttion_combiner_cAppHist(statis_partition_1,statis_partition_2 ))
        local_aggre_nodes= aggre_nodes.collect()
        cluster_end=time.time()
        
        #debug
        print len(local_aggre_nodes), current_NumNodes
    
        nodeSplits_layer=find_bestSplit_cAppHist(sorted(local_aggre_nodes,key= lambda val:val[0])\
                                    ,current_NumNodes,nodeSplits_tree,node_tree_test)
        #for whole tree
        currentNode_split_fromMaster = sc.broadcast(nodeSplits_tree)
        
    end = time.time()
    elapsed = end - start

    leaf_nodes = node_tree_test[ len(node_tree_test)-  (int)( math.pow(2,i)): len(node_tree_test)  ]
    
    tmp_test_err= tree_test( dta_test )
    tmp_train_err= tree_test( dta_train )
    
    #debug
    debug_tree_test( dta_test )
    
    test_err.append( tmp_test_err)
    train_err.append( tmp_train_err )
    run_time.append( elapsed )
    print "error at tree height", i,":",  tmp_test_err, tmp_train_err
    

1 1
error at tree height 1 : 761.202119717 765.22601824
1 1
2 2
error at tree height 2 : 737.481062936 740.863476202
1 1
2 2
4 4
error at tree height 3 : 711.817611255 715.673539037
1 1
2 2
4 4
8 8
error at tree height 4 : 697.67398466 700.485103145
1 1
2 2
4 4
8 8
16 16
error at tree height 5 : 508.113771835 509.441738697
1 1
2 2
4 4
8 8
16 16
32 32
error at tree height 6 : 182.829854839 181.065694693
1 1
2 2
4 4
8 8
16 16
32 32
64 64
error at tree height 7 : 552.75586191 558.05523979
1 1
2 2
4 4
8 8
16 16
32 32
64 64
75 128


IndexError: list index out of range